<a href="https://colab.research.google.com/github/ELehmann91/Thesis_Multilingual_Transferlearning/blob/master/Results_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train script
  
using prewritten fuctions and standardized data

In [50]:
import pandas as pd
from google.colab import drive, files

drive.mount('/content/gdrive')
path ='/content/gdrive/My Drive/Thesis_ecb_ecoicop'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%%capture
!pip install eli5
!git clone 'https://github.com/ELehmann91/Thesis_Multilingual_Transferlearning'

%cd Thesis_Multilingual_Transferlearning
import labeler_cc5
import coicop_model
import model_helper
import pandas as pd
import numpy as np
from tqdm import tqdm
import io

## Import Data
  
data in the normalized folder are splitted between languages (currently de fr & it) and share the same columns so they can be merged easily

In [51]:
import pandas as pd
import numpy as np

data_path = '/data/'#
#file_path = 'fra/carrfour_trans_pred.csv'
file_path = 'normalized/norm_fr.csv'
file_path2 = 'normalized/norm_de.csv'
file_path3 = 'at/norm_at.csv'
#file_path = 'edeka_pred.csv'

# french data
df_fr = pd.read_csv(path+data_path+file_path,sep='|',index_col=False)

# only already labeled
df_fr = df_fr[df_fr['cc5'].isna()==False]
df_fr = df_fr[df_fr['shop'].isin( ['carrefour','auchan'])]#,'banque_de_france'])]
df_fr['cc5'] = df_fr['cc5'].apply(lambda x: '9999_Non-Food' if int(str(x)[0])>2 else x)
print(len(df_fr))

# german data
df_de = pd.read_csv(path+data_path+file_path2,sep='|',index_col=False)
# only already labeled
df_de = df_de[df_de['cc5'].isna()==False]
print(len(df_de))
# austrian data
df_at = pd.read_csv(path+data_path+file_path3,sep='|',index_col=False)
# only already labeled
df_at = df_at[df_at['cc5'].isna()==False]
df_de = df_de.append(df_at)
print(len(df_de))

df_fr = df_fr.sample(frac=1).reset_index(drop=True)
df_de = df_de.sample(frac=1).reset_index(drop=True)

7923
21903
23199


### Exclude non-food?

In [52]:
no_Classes = 75
exclude_non_food = True
if exclude_non_food:
    df_fr = df_fr[df_fr['cc5']!='9999_Non-Food']
    df_de = df_de[df_de['cc5']!='9999_Non-Food']
    df_at = df_at[df_at['cc5']!='9999_Non-Food']
    no_Classes = 74
    print(len(df_fr))
    print(len(df_de))
    print(len(df_at))

7225
22064
241


decide which columns should be used for training

In [53]:
var = 'name'
#df_fr['text'] = df_fr[var].fillna('unknown') 
#df_de['text'] = df_de[var].fillna('unknown') 
#df_at['text'] = df_at[var].fillna('unknown') 

var1 = 'categ'
var2 = 'words_from_url'
#df_fr['text'] = df_fr[var1].fillna('unknown')  + ' <sep> ' + df_fr[var2].fillna('unknown') 
#df_de['text'] = df_de[var1].fillna('unknown')  + ' <sep> ' + df_de[var2].fillna('unknown') 
#df_at['text'] = df_at[var1].fillna('unknown')  + ' <sep> ' + df_at[var2].fillna('unknown') 


var1 = 'categ'
var2 = 'words_from_url' 
var3 = 'name'
df_fr['text'] = ' <fr> ' + df_fr[var1].fillna('unknown')  + ' <sep> ' + df_fr[var2].fillna('unknown')  + ' <sep> ' + df_fr[var3].fillna('unknown') 
df_de['text'] = ' <de> ' + df_de[var1].fillna('unknown')  + ' <sep> ' + df_de[var2].fillna('unknown')  + ' <sep> ' + df_de[var3].fillna('unknown') 
df_at['text'] = ' <de> ' + df_at[var1].fillna('unknown')  + ' <sep> ' + df_at[var2].fillna('unknown')  + ' <sep> ' + df_at[var3].fillna('unknown') 

#df_fr['text'] = df_fr[var1].fillna('unknown')  + df_fr[var2].fillna('unknown')  + df_fr[var3].fillna('unknown') 
#df_de['text'] = df_de[var1].fillna('unknown')  + df_de[var2].fillna('unknown')  + df_de[var3].fillna('unknown') 
#df_at['text'] = df_at[var1].fillna('unknown')  + df_at[var2].fillna('unknown')  + df_at[var3].fillna('unknown') 

#df_fr['text'] = ' <fr> ' + df_fr['name'] + ' <sep> ' + df_fr['categ'].fillna('unknown') + ' <sep> ' + df_fr['words_from_url'].fillna('unknown') + ' <sep> ' + df_fr['prod_desc'].fillna('unknown') 
#df_de['text'] = ' <de> ' + df_de['name'] + ' <sep> ' + df_de['categ'].fillna('unknown') + ' <sep> ' + df_de['words_from_url'].fillna('unknown') + ' <sep> ' + df_de['prod_desc'].fillna('unknown') 
#df_at['text'] = ' <de> ' + df_at['name'] + ' <sep> ' + df_at['categ'].fillna('unknown') + ' <sep> ' + df_at['words_from_url'].fillna('unknown') + ' <sep> ' + df_at['prod_desc'].fillna('unknown') 


In [54]:
import re
rep_dict = {'.':' ',
                        ',': ' ',
                        '&': ' ',
                        '-': ' ',
                        '/': ' ',
                        'ü': 'ue',
                        'ä': 'ae',
                        'ö': 'oe',
                        'ß': 'ss',
                        'ê': 'e',
                        'é': 'e',
                        'è': 'e',
                        'â': 'a',
                        'á': 'a',
                        'à': 'a',
                        'ô':'o',
                        'œ': 'ae',
                        '%': ' percent ',
                        '1': ' one ',
                        '2': ' two ',
                        '3': ' three ',
                        '4': ' four ',
                        '5': ' five ',
                        '6': ' six ',
                        '7': ' seven ',
                        '8': ' eigth ',
                        '9': ' nine ',
                        '0': ' zero ',
                        ' l ':' liter ',
                        ' ml ':' liter '
                        }

def prepro(line):
    if isinstance(line,str):
        text_str = ' '.join(str(t) for t in line.split())
        text_str = text_str.lower()
        for a,b in rep_dict.items():
            text_str = text_str.replace(a,b)
        text_str = re.sub('[^a-zäöüàáâéèêßœ<>]+', ' ', text_str)
    else: 
        text_str = str(line)
    return text_str

In [55]:
df_fr['text'] = df_fr['text'].apply(lambda x:prepro(x))
df_de['text'] = df_de['text'].apply(lambda x:prepro(x))
df_at['text'] = df_at['text'].apply(lambda x:prepro(x))

In [56]:
print('50% quantile no. of words per row french',np.quantile(df_fr['text'].apply(lambda x: len(str(x).split())),.50))
print('50% quantile no. of words per row german',np.quantile(df_de['text'].apply(lambda x: len(str(x).split())),.50))
print('50% quantile no. of words per row austrian',np.quantile(df_at['text'].apply(lambda x: len(str(x).split())),.50))

print('90% quantile no. of words per row french',np.quantile(df_fr['text'].apply(lambda x: len(str(x).split())),.90))
print('90% quantile no. of words per row german',np.quantile(df_de['text'].apply(lambda x: len(str(x).split())),.90))
print('90% quantile no. of words per row austrian',np.quantile(df_at['text'].apply(lambda x: len(str(x).split())),.90))

seq_len = int(max(np.quantile(df_fr['text'].apply(lambda x: len(str(x).split())),.95),np.quantile(df_de['text'].apply(lambda x: len(str(x).split())),.95)))
print('seq_len',seq_len)

50% quantile no. of words per row french 28.0
50% quantile no. of words per row german 19.0
50% quantile no. of words per row austrian 19.0
90% quantile no. of words per row french 36.0
90% quantile no. of words per row german 29.700000000000728
90% quantile no. of words per row austrian 24.0
seq_len 39


### Split Train Test

In [57]:
from sklearn.model_selection import train_test_split
def split_train_abs(df):
    X_train, X_val_test, y_train, y_val_test  = train_test_split(df['text'], df['cc5'], random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test , y_val_test, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train_de, X_val_de, X_test_de, y_train_de, y_val_de, y_test_de = split_train_abs(df_de)
X_train_fr, X_val_fr, X_test_fr, y_train_fr, y_val_fr, y_test_fr = split_train_abs(df_fr)

print('de',X_train_de.shape, X_val_de.shape, X_test_de.shape, y_train_de.shape, y_val_de.shape, y_test_de.shape )
print('fr',X_train_fr.shape, X_val_fr.shape, X_test_fr.shape, y_train_fr.shape, y_val_fr.shape, y_test_fr.shape )

de (16548,) (4137,) (1379,) (16548,) (4137,) (1379,)
fr (5418,) (1355,) (452,) (5418,) (1355,) (452,)


In [58]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer_de = Tokenizer()
tokenizer_de.fit_on_texts(X_train_de.append(X_val_de))
vocab_size_de = len(tokenizer_de.word_index) + 1

tokenizer_fr = Tokenizer()
tokenizer_fr.fit_on_texts(X_train_fr.append(X_val_fr))
vocab_size_fr = len(tokenizer_fr.word_index) + 1

tokenizer_de_fr = Tokenizer()
tokenizer_de_fr.fit_on_texts(X_train_de.append(X_val_de).append(X_train_fr).append(X_val_fr))

vocab_size_de_fr = len(tokenizer_de_fr.word_index) + 1
print(vocab_size_de,vocab_size_fr,vocab_size_de_fr)

12590 3991 15261


tokenize & pad

In [59]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#german
X_train_tokens_de = tokenizer_de.texts_to_sequences(X_train_de)
X_val_tokens_de = tokenizer_de.texts_to_sequences(X_val_de)
X_test_tokens_de = tokenizer_de.texts_to_sequences(X_test_de)

X_train_pad_de = pad_sequences(X_train_tokens_de,maxlen=seq_len, padding='post')
X_val_pad_de = pad_sequences(X_val_tokens_de,maxlen=seq_len, padding='post')
X_test_pad_de = pad_sequences(X_test_tokens_de,maxlen=seq_len, padding='post')

#french
X_train_tokens_fr = tokenizer_fr.texts_to_sequences(X_train_fr)
X_val_tokens_fr = tokenizer_fr.texts_to_sequences(X_val_fr)
X_test_tokens_fr = tokenizer_fr.texts_to_sequences(X_test_fr)

X_train_pad_fr = pad_sequences(X_train_tokens_fr,maxlen=seq_len, padding='post')
X_val_pad_fr = pad_sequences(X_val_tokens_fr,maxlen=seq_len, padding='post')
X_test_pad_fr = pad_sequences(X_test_tokens_fr,maxlen=seq_len, padding='post')

In [60]:
import pickle
french = True
german = True

embedding_dim = 300

if french:
    fr_git_embed = pickle.load( open(path + '/embeddings/fr_slim_embed_ext.p', "rb" ) ) #fr_slim_embed_ext #fr_muse_align #fr_muse
    #fr_model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/Thesis_ecb_ecoicop/embeddings/wiki.fr.vec')
if german:
    de_git_embed = pickle.load( open(path + '/embeddings/de_slim_embed_ext.p', "rb" ) ) #de_muse
    #de_model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/Thesis_ecb_ecoicop/embeddings/wiki.de.vec')

v = np.zeros(300)
v[0]=1
fr_git_embed['<sep>'] = v
de_git_embed['<sep>'] = v
v[0]=0
v[1]=1
fr_git_embed['<fr>'] = v
v[1]=-1
de_git_embed['<de>'] = v

print('de_git_embed',len(de_git_embed.keys()))
print('fr_git_embed',len(fr_git_embed.keys()))

X_train_emb_de = np.array(list(model_helper.text_to_embed(X_train_de, ['de' for a in X_train_de], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_val_emb_de = np.array(list(model_helper.text_to_embed(X_val_de, ['de' for a in X_val_de], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_test_emb_de = np.array(list(model_helper.text_to_embed(X_test_de, ['de' for a in X_test_de], de_git_embed, fr_git_embed, seq_len=seq_len)))

X_train_emb_fr = np.array(list(model_helper.text_to_embed(X_train_fr, ['fr' for a in X_train_fr], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_val_emb_fr = np.array(list(model_helper.text_to_embed(X_val_fr, ['fr' for a in X_val_fr], de_git_embed, fr_git_embed, seq_len=seq_len)))
X_test_emb_fr = np.array(list(model_helper.text_to_embed(X_test_fr, ['fr' for a in X_test_fr], de_git_embed, fr_git_embed, seq_len=seq_len)))

print('de',X_train_emb_de.shape, X_val_emb_de.shape, X_test_emb_de.shape)
print('fr',X_train_emb_fr.shape, X_val_emb_fr.shape, X_test_emb_fr.shape)

664it [00:00, 6639.42it/s]

de_git_embed 64413
fr_git_embed 43014


16548it [00:02, 6158.69it/s]
4137it [00:00, 6478.17it/s]
1379it [00:00, 5994.88it/s]
5418it [00:00, 6822.64it/s]
1355it [00:00, 6612.09it/s]
452it [00:00, 6596.91it/s]


de (16548, 39, 300) (4137, 39, 300) (1379, 39, 300)
fr (5418, 39, 300) (1355, 39, 300) (452, 39, 300)


In [61]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

encoder = LabelEncoder()
encoder.fit(df_de['cc5'])

def encode_label(y_):
    y__ = encoder.transform(y_)
    y_enc =tf.keras.utils.to_categorical(y__, num_classes=no_Classes, dtype="float32")
    return y_enc

y_train_enc_de = encode_label(y_train_de)
y_val_enc_de = encode_label(y_val_de)
y_test_enc_de = encode_label(y_test_de)

y_train_enc_fr = encode_label(y_train_fr)
y_val_enc_fr = encode_label(y_val_fr)
y_test_enc_fr = encode_label(y_test_fr)

print(y_train_enc_de.shape,y_val_enc_de.shape,y_test_enc_de.shape)
print(y_train_enc_fr.shape,y_val_enc_fr.shape,y_test_enc_fr.shape)

(16548, 74) (4137, 74) (1379, 74)
(5418, 74) (1355, 74) (452, 74)


In [62]:
def get_weigth_dict(y_train):
    weights_dict = dict(zip(y_train.value_counts().index.tolist(),list(len(y_train) / ( len(y_train.unique())  * y_train.value_counts()))))

    class_weight_dict = {}
    for n,lab in enumerate(encoder.classes_):
        try:
            class_weight_dict[n] = weights_dict[lab]
        except:
            class_weight_dict[n] = 1
    return class_weight_dict

class_weight_dict_de = get_weigth_dict(y_train_de)
class_weight_dict_fr = get_weigth_dict(y_train_fr)
class_weight_dict_de_fr = get_weigth_dict(y_train_de.append(y_train_fr))

In [64]:
unknown = np.random.rand(embedding_dim)

def get_embed_matrix(embed,tokenizer,vocab_size,embedding_dim):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word,i in tokenizer.word_index.items():
        try:
            embedding_matrix[i] = embed[word]
        except KeyError:
            #next
            embedding_matrix[i] = unknown #np.random.rand(embedding_dim)
    return embedding_matrix

embedding_matrix_de = get_embed_matrix(de_git_embed,tokenizer_de,vocab_size_de,embedding_dim)
embedding_matrix_fr = get_embed_matrix(fr_git_embed,tokenizer_fr,vocab_size_fr,embedding_dim)

combine_embed = de_git_embed
combine_embed.update(fr_git_embed)
embedding_matrix_de_fr = get_embed_matrix(combine_embed,tokenizer_de_fr,vocab_size_de_fr,embedding_dim)


# Single Language Classifier German

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression

In [17]:
def tf_idf_log_reg(X_train,y_train,X_test,y_test,no,weights_dict):
    X_train = [str(x).replace('<','').replace('>','') for x in X_train[:no]]
    y_train = y_train[:no]
    X_test = [str(x).replace('<','').replace('>','') for x in X_test]
    vectorizer  = TfidfVectorizer()
    vectorizer.fit(X_train)
    X_train_vec = vectorizer.transform(X_train)
    X_test_vec  = vectorizer.transform(X_test)

    logreg = LogisticRegression(C=1,max_iter=500, solver='newton-cg')#,class_weight=weights_dict)
    logreg.fit(X_train_vec, y_train)

    y_pred_train = logreg.predict(X_train_vec)
    y_pred_test = logreg.predict(X_test_vec)

    print('train obs.',no,'accuracy %s' % accuracy_score(y_pred_train, y_train))
    print('train obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_train, y_train))
    print('test obs.',no,'accuracy %s' % accuracy_score(y_pred_test, y_test))
    print('test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test, y_test))
    print()

##LogReg

In [18]:
tf_idf_log_reg(X_train_de,y_train_de,X_test_de,y_test_de,7500,class_weight_dict_de)

train obs. 7500 accuracy 0.948
train obs. 7500 b_accuracy 0.9554857711222235
test obs. 7500 accuracy 0.9093546047860769
test obs. 7500 b_accuracy 0.9008351224575238



In [19]:
stra = False
for obs in [250,500,1000,2000,5000,10000,15000]:
    tf_idf_log_reg(X_train_de,y_train_de,X_test_de,y_test_de,obs,class_weight_dict_de)

train obs. 250 accuracy 0.728
train obs. 250 b_accuracy 0.9227584603585467
test obs. 250 accuracy 0.3727338651196519
test obs. 250 b_accuracy 0.7807774081653794

train obs. 500 accuracy 0.754
train obs. 500 b_accuracy 0.9356016875681188
test obs. 500 accuracy 0.48223350253807107
test obs. 500 b_accuracy 0.819003535956219

train obs. 1000 accuracy 0.861
train obs. 1000 b_accuracy 0.944278209737142
test obs. 1000 accuracy 0.6932559825960841
test obs. 1000 b_accuracy 0.8707367800131294

train obs. 2000 accuracy 0.9015
train obs. 2000 b_accuracy 0.9393502431781049
test obs. 2000 accuracy 0.8158085569253082
test obs. 2000 b_accuracy 0.8771264885273545

train obs. 5000 accuracy 0.9392
train obs. 5000 b_accuracy 0.9484502782358264
test obs. 5000 accuracy 0.8905003625815808
test obs. 5000 b_accuracy 0.9027686807727505

train obs. 10000 accuracy 0.9535
train obs. 10000 b_accuracy 0.957200699258984
test obs. 10000 accuracy 0.9187817258883249
test obs. 10000 b_accuracy 0.9035734622541813

train o

## NN Definition

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense,Embedding, Flatten,Reshape, GlobalAveragePooling1D,MaxPool2D, Concatenate,Conv1D,Conv2D,BatchNormalization,Add,Masking,GlobalMaxPooling1D,LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.regularizers import l1,l2
import pickle
import numpy as np

In [21]:
def run_CNN(X_train_pad,y_train_enc,X_val_pad,y_val_enc,X_test_pad,y_test,vocab_size,embedding_matrix,class_weight_dict,no):
    
    dropout_rate=.5
    lr = .005
    opt = Adam(lr=lr, decay=lr/75)

    input_layer = Input(shape = (seq_len,), dtype = 'float')
    embedd_seq = Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], input_length = seq_len, trainable = True, mask_zero=False)(input_layer)
    conv_layer = Conv1D(filters=100,   kernel_size= 5,   padding='valid',  activation='relu', strides=1)(embedd_seq)
    pool_layer = GlobalMaxPooling1D()(conv_layer)
    #pool_layer = GlobalAveragePooling1D()(embedd_seq)
    drop_layer = Dropout(dropout_rate)(pool_layer)
    pred_layer = Dense(y_train_enc.shape[1], activation = 'softmax')(drop_layer) #, kernel_regularizer=l1_l2(0.001)

    avg_pool_mod = Model(inputs = [input_layer], outputs = pred_layer)
    avg_pool_mod.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = avg_pool_mod.fit(x = X_train_pad[:no], y = y_train_enc[:no],\
                    validation_data = (X_val_pad, y_val_enc), \
                    epochs = 50, batch_size = 256, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test = avg_pool_mod.predict(X_test_pad)
    y_pred_arg = y_pred_test.argmax(axis=1)
    y_pred_test= [encoder.classes_[y] for y in y_pred_arg]

    print('test obs.',no,'accuracy %s' % accuracy_score(y_pred_test, y_test))
    print('test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test, y_test))


In [28]:
def run_CNN2D(X_train_pad,y_train_enc,X_val_pad,y_val_enc,X_test_pad,y_test,vocab_size,embedding_matrix,class_weight_dict,no):
    
    dropout_rate= .5
    filter_sizes = [1,3,5,7,9,11,13]
    num_filters = 11

    input_layer = Input(shape = (seq_len,), name='text_input')
    x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable = True)(input_layer)
    x = Reshape((seq_len, embedding_dim, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_dim),
                                      kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(seq_len - filter_sizes[i] + 1, 1))(conv))

    pool_layer = Concatenate(axis=1)(maxpool_pool)   

    #conv_layer = Conv2D(filters=100,   kernel_size=(5,300),   padding='same',  activation='relu', strides=1,name='convolution')(x)
    #pool_layer = MaxPool2D(pool_size = (35,1),name='max_pooling')(conv_layer)
    falt_layer = Flatten(name='flat')(pool_layer)
    drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn2d = Model(inputs = [input_layer], outputs = [pred_layer])
    #de_cnn2d.summary()
    lr = .005
    opt = Adam(lr=lr, decay=lr/100)

    de_cnn2d.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = de_cnn2d.fit(x = X_train_pad[:no], y = y_train_enc[:no],\
                    validation_data = (X_val_pad, y_val_enc), \
                    epochs = 100, batch_size = 256, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test = de_cnn2d.predict(X_test_pad)
    y_pred_arg = y_pred_test.argmax(axis=1)
    y_pred_test= [encoder.classes_[y] for y in y_pred_arg]

    print('test obs.',no,'accuracy %s' % accuracy_score(y_pred_test, y_test))
    print('test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test, y_test))

In [23]:
def run_RNN(X_train_pad,y_train_enc,X_val_pad,y_val_enc,X_test_pad,y_test,vocab_size,embedding_matrix,class_weight_dict,no):
    dropout_rate=.5
    lr = .005
    opt = Adam(lr=lr, decay=lr/100)

    input_layer = Input(shape = (seq_len,), dtype = 'int32')
    embedd_seq = Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], input_length = seq_len, trainable = True)(input_layer)
    lstm_layer = Bidirectional(LSTM(128,activation = 'tanh',recurrent_activation = 'sigmoid', dropout = dropout_rate,recurrent_dropout = 0, unroll = False,use_bias = True))(embedd_seq)
    drop_layer = Dropout(dropout_rate)(lstm_layer)
    pred_layer = Dense(y_train_enc.shape[1], activation = 'softmax')(drop_layer) #, kernel_regularizer=l1_l2(0.001)

    avg_pool_mod = Model(inputs = [input_layer], outputs = pred_layer)
    avg_pool_mod.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = avg_pool_mod.fit(x = X_train_pad[:no], y = y_train_enc[:no],\
                    validation_data = (X_val_pad, y_val_enc), \
                    epochs = 50, batch_size = 256, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test = avg_pool_mod.predict(X_test_pad)
    y_pred_arg = y_pred_test.argmax(axis=1)
    y_pred_test= [encoder.classes_[y] for y in y_pred_arg]

    print('test obs.',no,'accuracy %s' % accuracy_score(y_pred_test, y_test))
    print('test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test, y_test))
    print()

##CNN

In [26]:
for obs in [500,1000,2000,5000,10000,15000]:
     run_CNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,vocab_size_de,embedding_matrix_de,class_weight_dict_de,obs)


Epoch 1/50
2/2 [==============================] - 1s 271ms/step - loss: 4.2286 - accuracy: 0.0140 - val_loss: 4.2940 - val_accuracy: 0.0544
Epoch 2/50
2/2 [==============================] - 0s 158ms/step - loss: 3.9401 - accuracy: 0.0360 - val_loss: 4.2567 - val_accuracy: 0.0051
Epoch 3/50
2/2 [==============================] - 0s 130ms/step - loss: 3.6723 - accuracy: 0.0340 - val_loss: 4.1654 - val_accuracy: 0.0213
Epoch 4/50
2/2 [==============================] - 0s 130ms/step - loss: 3.4897 - accuracy: 0.0520 - val_loss: 4.0852 - val_accuracy: 0.0771
Epoch 5/50
2/2 [==============================] - 0s 129ms/step - loss: 3.2847 - accuracy: 0.1160 - val_loss: 4.0239 - val_accuracy: 0.0935
Epoch 6/50
2/2 [==============================] - 0s 122ms/step - loss: 3.0997 - accuracy: 0.1680 - val_loss: 3.9226 - val_accuracy: 0.0844
Epoch 7/50
2/2 [==============================] - 0s 125ms/step - loss: 2.9898 - accuracy: 0.2020 - val_loss: 3.8275 - val_accuracy: 0.1160
Epoch 8/50
2/2 [====

KeyboardInterrupt: ignored

## CNN2D

In [29]:
for obs in [100,250,500,1000,2000,5000,10000,15000]:
     run_CNN2D(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,vocab_size_de,embedding_matrix_de,class_weight_dict_de,obs)


Epoch 1/100
1/1 [==============================] - 1s 589ms/step - loss: 3.5526 - accuracy: 0.0000e+00 - val_loss: 4.4865 - val_accuracy: 0.0295
Epoch 2/100
1/1 [==============================] - 0s 413ms/step - loss: 3.5687 - accuracy: 0.0000e+00 - val_loss: 4.4419 - val_accuracy: 0.0237
Epoch 3/100
1/1 [==============================] - 0s 412ms/step - loss: 3.0838 - accuracy: 0.0600 - val_loss: 4.4358 - val_accuracy: 0.0329
Epoch 4/100
1/1 [==============================] - 0s 407ms/step - loss: 2.8055 - accuracy: 0.0900 - val_loss: 4.4048 - val_accuracy: 0.0220
Epoch 5/100
1/1 [==============================] - 0s 415ms/step - loss: 2.9749 - accuracy: 0.0800 - val_loss: 4.3893 - val_accuracy: 0.0143
Epoch 6/100
1/1 [==============================] - 1s 721ms/step - loss: 2.7339 - accuracy: 0.0900 - val_loss: 4.3398 - val_accuracy: 0.0230
Epoch 7/100
1/1 [==============================] - 0s 412ms/step - loss: 2.3966 - accuracy: 0.1700 - val_loss: 4.2910 - val_accuracy: 0.0365
Epoch

## RNN

In [ ]:
for obs in [500,1000,2000,5000,10000,15000]:
    run_RNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,vocab_size_de,embedding_matrix_de,class_weight_dict_de,obs)

## Best Model

In [ ]:
%load_ext tensorboard
import os, datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
 
def run_CNN_best(X_train_pad,y_train_enc,X_val_pad,y_val_enc,X_test_pad,y_test,vocab_size,embedding_matrix,class_weight_dict):
    
    dropout_rate=.7
    lr = .0005
    opt = Adam(lr=lr, decay=lr/75)

    input_layer = Input(shape = (seq_len,), dtype = 'float',name='text_input')
    embedd_seq = Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], input_length = seq_len, trainable = True, mask_zero=False,name='muse_embedding')(input_layer)
    conv_layer = Conv1D(filters=150,   kernel_size= 5,   padding='valid',  activation='relu', strides=1,name='1d_convolution')(embedd_seq)
    pool_layer = GlobalMaxPooling1D(name='max_pooling')(conv_layer)
    #pool_layer = GlobalAveragePooling1D()(embedd_seq)
    drop_layer = Dropout(dropout_rate,name='dropout')(pool_layer)
    pred_layer = Dense(y_train_enc.shape[1], activation = 'softmax',name='dense_prediction')(drop_layer) #, kernel_regularizer=l1_l2(0.001)

    cnn_de_fin = Model(inputs = [input_layer], outputs = pred_layer)
    cnn_de_fin.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)
    print(cnn_de_fin.summary())
    hist = cnn_de_fin.fit(x = X_train_pad, y = y_train_enc,\
                    validation_data = (X_val_pad, y_val_enc), \
                    epochs = 50, batch_size = 64, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test = cnn_de_fin.predict(X_test_pad)
    y_pred_arg = y_pred_test.argmax(axis=1)
    y_pred_test= [encoder.classes_[y] for y in y_pred_arg]

    print('test obs.','accuracy %s' % accuracy_score(y_pred_test, y_test))
    print('test obs.','b_accuracy %s' % balanced_accuracy_score(y_pred_test, y_test))


In [ ]:
run_CNN_best(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,vocab_size_de,embedding_matrix_de,class_weight_dict_de)

In [ ]:
%tensorboard --logdir logs

# Single Language Classifier French

##LogReg

In [ ]:
stra = False
for obs in [250,500,1000,2000,5000]:
    tf_idf_log_reg(X_train_de,y_train_de,X_test_de,y_test_de,obs,class_weight_dict_de)

##CNN

In [ ]:
for obs in [500,1000,2000,5000]:
     run_CNN(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,vocab_size_fr,embedding_matrix_fr,class_weight_dict_fr,obs)


## CNN2D

In [ ]:
for obs in [100,250,500,1000,2000,5000]:
     run_CNN2D(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,vocab_size_fr,embedding_matrix_fr,class_weight_dict_fr,obs)


## RNN

In [ ]:
for obs in [500,1000,2000,5000]:
     run_RNN(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,vocab_size_fr,embedding_matrix_fr,class_weight_dict_fr,obs)


## Best Model

In [ ]:
run_CNN_best(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,vocab_size_fr,embedding_matrix_fr,class_weight_dict_fr)

# Multilingual Transfer

load embedings  
we only load a "slim" version of the embeddings, which are a subset of the vocab (less than 5%) the time loading the embeddings decreases from 15 min to 7 sec and colab is capable of loading more than two languages (embedding)

### Co2

In [ ]:
#!pip install experiment-impact-tracker
#!mkdir logs
#from experiment_impact_tracker.compute_tracker import ImpactTracker
#tracker = ImpactTracker('logs')
#tracker.launch_impact_monitor()
#info = tracker.get_latest_info_and_check_for_errors()
#!ls logs/impacttracker
#!cat logs/impacttracker/impact_tracker_log.log
#!wget 'https://raw.githubusercontent.com/ELehmann91/Thesis_Multilingual_Transferlearning/master/data/model.json'
!create-compute-appendix logs/ --site_spec model.json --output_dir logs/

### LogReg

In [ ]:
np.mean(X_train_emb_de,axis=1).shape

In [ ]:
def tf_idf_log_reg(X_train,y_train,X_test,y_test,no):
    X_train_vec = np.mean(X_train[:no],axis=1)
    y_train = y_train[:no]
    X_test_vec = np.mean(X_test,axis=1)

    logreg = LogisticRegression(C=0.9,max_iter=100, solver='saga',penalty='elasticnet',l1_ratio=.1)
    logreg.fit(X_train_vec, y_train)

    y_pred_train = logreg.predict(X_train_vec)
    y_pred_test = logreg.predict(X_test_vec)

    print('train obs.',no,'accuracy %s' % accuracy_score(y_pred_train, y_train))
    print('train obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_train, y_train))
    print('test obs.',no,'accuracy %s' % accuracy_score(y_pred_test, y_test))
    print('test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test, y_test))
    print()


In [ ]:
tf_idf_log_reg(X_train_emb_de,y_train_de,X_test_emb_fr,y_test_fr,30000)

## CNN 1D

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Bidirectional,LSTM, GlobalAveragePooling1D, Concatenate,Conv1D,BatchNormalization,Add,Masking,GlobalMaxPooling1D,LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.regularizers import l1,l2
dropout_rate=.1

input_layer = Input(shape = (seq_len,embedding_dim,), name='text_input')
pool_layer = GlobalAveragePooling1D(name='avg_pooling')(input_layer)
#dens_layer = Dense(155, activation='relu',name='dense150')(pool_layer)
drop_layer = Dropout(dropout_rate,name='drop')(pool_layer)
pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

de_avg_pool = Model(inputs = [input_layer], outputs = [pred_layer])
de_avg_pool.summary()
lr = .005
opt = Adam(lr=lr, decay=lr/100)

de_avg_pool.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)

hist = de_avg_pool.fit(x = X_train_emb_de, y = y_train_enc_de,\
                validation_data = (X_val_emb_de, y_val_enc_de), \
                epochs = 100, batch_size = 256, shuffle = True, \
                class_weight = class_weight_dict_de, \
                callbacks = [early_stopping])

In [ ]:
import json
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,balanced_accuracy_score,confusion_matrix
import numpy as np

def test_xy(X,y,string,model):
    y_pred = model.predict([X])
    y_pred_arg = y_pred.argmax(axis=1)
    pred= [encoder.classes_[y] for y in y_pred_arg]
    print('accuracy %s'% string,accuracy_score(pred,y))
    print('b_accuracy %s'%  string,balanced_accuracy_score(pred, y))


test_xy(X_test_emb_de,y_test_de,'german',de_avg_pool)
test_xy(X_test_emb_fr,y_test_fr,'french',de_avg_pool)
test_xy(X_val_emb_fr,y_val_fr,'french',de_avg_pool)
test_xy(X_train_emb_fr,y_train_fr,'french',de_avg_pool)

### deeper prediction analysis

Prediction / Certainty Treshold

In [ ]:
def test_xy(X,y,string,model,z,ret=False):
    obs = len(X)
    y_pred = model.predict([X])
    y_pred_arg = y_pred.argmax(axis=1)
    y_pred_max = y_pred.max(axis=1)
    pred= [encoder.classes_[y] for y in y_pred_arg]
    df_pred = pd.DataFrame(zip(X,pred,y,y_pred_max),columns=['embed','pred','y','y_pred_max'])
    df_pred = df_pred[df_pred['y_pred_max']>z]
    print('accuracy %s'% string,accuracy_score(df_pred['pred'],df_pred['y']))
    print('b_accuracy %s'%  string,balanced_accuracy_score(df_pred['pred'],df_pred['y']))
    print(len(df_pred),'of',len(X),int(len(df_pred)/len(X)*100),'%',len(df_pred.pred.unique()))
    if ret:
        return df_pred
test_xy(X_test_emb_de,y_test_de,'german',de_avg_pool,.65)
test_xy(X_traprint(classification_report(y,pred))in_emb_fr,y_train_fr,'french',de_avg_pool,.8)
    #return pred, y

In [ ]:
print(classification_report(y,pred))

Accuracy for higher hirachies

In [ ]:
with open('coicop_5_4.txt') as json_file:#
    coicop_5_4 = json.load(json_file)

with open('coicop_5_3.txt') as json_file:#
    coicop_5_3 = json.load(json_file)

y_pred = de_avg_pool.predict([X_test_emb_fr])
y_pred_arg = y_pred.argmax(axis=1)
pred= [encoder.classes_[y] for y in y_pred_arg]

y_pr_lab3 = [coicop_5_3[cc5] for cc5 in pred]
y_pr_lab4 = [coicop_5_4[cc5] for cc5 in pred]
y_lab3 = [coicop_5_3[cc5] for cc5 in y_test_fr]
y_lab4 = [coicop_5_4[cc5] for cc5 in y_test_fr]
print('accuracy %s'% accuracy_score(y_pr_lab3,y_lab3))
print('b_accuracy %s'%  balanced_accuracy_score(y_pr_lab3, y_lab3))
print('accuracy %s'% accuracy_score(y_pr_lab4,y_lab4))
print('b_accuracy %s'%  balanced_accuracy_score(y_pr_lab4, y_lab4))
#print(classification_report(y_lab4,y_pr_lab4))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

y_pred = de_avg_pool.predict([X_test_emb_fr])
y_pred_arg = y_pred.argmax(axis=1)
pred= [encoder.classes_[y] for y in y_pred_arg]

y_pr_lab4 = [coicop_5_4[cc5] for cc5 in pred]
y_lab4 = [coicop_5_4[cc5] for cc5 in y_test_fr]

label = pd.Series(y_lab4).unique()
label.sort()
cm = confusion_matrix(y_lab4,y_pr_lab4,labels=label)#, normalize='true')

df_cm = pd.DataFrame(cm, label,label)
plt.figure(figsize=(15,15))
sn.set(font_scale=.7) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 9}) # font size

plt.show()

### Transferlearning

In [ ]:
few_shot = 1000

lr = .005
opt = Adam(lr=lr, decay=lr/100)
inp = de_avg_pool.input
out = de_avg_pool.output #get_layer('output_de').output

# create a new network between inp and out
model_new = Model(inp, out)
model_new.summary()
model_new.get_layer('convolution').trainable = False
model_new.get_layer('drop').rate = .5

model_new.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)

model_new.fit(x = X_train_emb_fr[:few_shot], y = y_train_enc_fr[:few_shot],\
                validation_data = (X_val_emb_fr, y_val_enc_fr), \
                epochs = 100, batch_size = 32, shuffle = True, \
                class_weight = class_weight_dict_fr, \
                callbacks = [early_stopping])

In [ ]:
test_xy(X_test_emb_de,y_test_de,'german',model_new)
test_xy(X_test_emb_fr,y_test_fr,'french',model_new)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

y_pred = model_new.predict([X_test_emb_fr])
y_pred_arg = y_pred.argmax(axis=1)
pred= [encoder.classes_[y] for y in y_pred_arg]

y_pr_lab4 = [coicop_5_4[cc5] for cc5 in pred]
y_lab4 = [coicop_5_4[cc5] for cc5 in y_test_fr]
print('accuracy %s'% accuracy_score(y_pr_lab4,y_lab4))
print('b_accuracy %s'%  balanced_accuracy_score(y_pr_lab4, y_lab4))

label = pd.Series(y_test_fr).unique()
label.sort()
cm = confusion_matrix(y_test_fr,pred,labels=label)#, normalize='true')

df_cm = pd.DataFrame(cm, label,label)
plt.figure(figsize=(15,15))
sn.set(font_scale=.7) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 9}) # font size

plt.show()

In [ ]:
test_xy(X_test_emb_de,y_test_de,'german',de_avg_pool_2)
test_xy(X_test_emb_fr,y_test_fr,'french',de_avg_pool_2)

In [ ]:
#print(classification_report(pred, df_acc['cc5']))

## CCN 2D

In [91]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten,Reshape, GlobalAveragePooling1D,MaxPool2D,AvgPool2D, Concatenate,Conv1D,Conv2D,BatchNormalization,Add,Masking,GlobalMaxPooling1D,LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.regularizers import l1,l2
dropout_rate=.35
filter_sizes = [1,2,3,4,5]
num_filters = 15

input_layer = Input(shape = (seq_len,embedding_dim,), name='text_input')
x = Reshape((seq_len, embedding_dim, 1))(input_layer)

maxpool_pool = []
for i in range(len(filter_sizes)):
    conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_dim),
                                  kernel_initializer='he_normal', activation='relu')(x)
    maxpool_pool.append(AvgPool2D(pool_size=(seq_len - filter_sizes[i] + 1, 1))(conv))

pool_layer = Concatenate(axis=1)(maxpool_pool)   

#conv_layer = Conv2D(filters=100,   kernel_size=(5,300),   padding='same',  activation='relu', strides=1,name='convolution')(x)
#pool_layer = MaxPool2D(pool_size = (35,1),name='max_pooling')(conv_layer)
falt_layer = Flatten(name='flat')(pool_layer)
drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

de_cnn2d = Model(inputs = [input_layer], outputs = [pred_layer])
de_cnn2d.summary()
lr = .005
opt = Adam(lr=lr, decay=lr/150)

de_cnn2d.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)

hist = de_cnn2d.fit(x = X_train_emb_de, y = y_train_enc_de,\
                validation_data = (X_val_emb_de, y_val_enc_de), \
                epochs = 150, batch_size = 256, shuffle = True, \
                class_weight = class_weight_dict_de, \
                callbacks = [early_stopping])

Model: "model_65"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 39, 300)]    0                                            
__________________________________________________________________________________________________
reshape_22 (Reshape)            (None, 39, 300, 1)   0           text_input[0][0]                 
__________________________________________________________________________________________________
conv2d_134 (Conv2D)             (None, 39, 1, 15)    4515        reshape_22[0][0]                 
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 38, 1, 15)    9015        reshape_22[0][0]                 
___________________________________________________________________________________________

In [92]:
def test_xy(X,y,string,model):
    y_pred = model.predict([X])
    y_pred_arg = y_pred.argmax(axis=1)
    pred= [encoder.classes_[y] for y in y_pred_arg]
    print('accuracy %s'% string,accuracy_score(pred,y))
    print('b_accuracy %s'%  string,balanced_accuracy_score(pred, y))


test_xy(X_test_emb_de,y_test_de,'german',de_cnn2d)
test_xy(X_test_emb_fr,y_test_fr,'french',de_cnn2d)
test_xy(X_val_emb_fr,y_val_fr,'french',de_cnn2d)
test_xy(X_train_emb_fr,y_train_fr,'french',de_cnn2d)

accuracy german 0.9506889050036258
b_accuracy german 0.8980103969752893
accuracy french 0.3915929203539823
b_accuracy french 0.40650398303962376
accuracy french 0.38671586715867157
b_accuracy french 0.44627046134971154
accuracy french 0.39516426725729054
b_accuracy french 0.41444340959734666


### Transferlearning

In [87]:
def transfer_cnn(few_shot,freeze):

    lr = .005
    opt = Adam(lr=lr, decay=lr/100)
    inp = de_cnn2d.input
    out = de_cnn2d.output #get_layer('output_de').output

    # create a new network between inp and out
    de_cnn2d_transfer = Model(inp, out)
    #de_cnn2d_transfer.summary()
    #de_cnn2d_transfer.get_layer('flat').trainable = False
    #de_cnn2d_transfer.get_layer('drop').rate = .6
    

    de_cnn2d_transfer.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)
    #freeze
    for l , layer in enumerate(de_cnn2d_transfer.layers[:-3]):
        de_cnn2d_transfer.layers[l].trainable = (freeze==False)

    de_cnn2d_transfer.fit(x = X_train_emb_fr[:few_shot], y = y_train_enc_fr[:few_shot],\
                    validation_data = (X_val_emb_fr, y_val_enc_fr), \
                    epochs = 100, batch_size = 32, shuffle = True, \
                    class_weight = class_weight_dict_fr, \
                    callbacks = [early_stopping])
    print(few_shot)
    print(test_xy(X_test_emb_de,y_test_de,'german',de_cnn2d_transfer))
    print(test_xy(X_test_emb_fr,y_test_fr,'french test',de_cnn2d_transfer))
    print(test_xy(X_val_emb_fr,y_val_fr,'french val',de_cnn2d_transfer))
    print(test_xy(X_train_emb_fr,y_train_fr,'french train',de_cnn2d_transfer))

In [79]:
for obs in [100,250,500,1000,2000,5000]:
    transfer_cnn(obs,True)

Epoch 1/100
4/4 [==============================] - 1s 234ms/step - loss: 7.7485 - accuracy: 0.2300 - val_loss: 3.9185 - val_accuracy: 0.4590
Epoch 2/100
4/4 [==============================] - 0s 60ms/step - loss: 4.8554 - accuracy: 0.3300 - val_loss: 3.7313 - val_accuracy: 0.4590
Epoch 3/100
4/4 [==============================] - 0s 61ms/step - loss: 3.5603 - accuracy: 0.3900 - val_loss: 3.6592 - val_accuracy: 0.4524
Epoch 4/100
4/4 [==============================] - 0s 60ms/step - loss: 3.3046 - accuracy: 0.3900 - val_loss: 3.6225 - val_accuracy: 0.4465
Epoch 5/100
4/4 [==============================] - 0s 58ms/step - loss: 2.4936 - accuracy: 0.4800 - val_loss: 3.6462 - val_accuracy: 0.4384
Epoch 6/100
4/4 [==============================] - 0s 57ms/step - loss: 2.8111 - accuracy: 0.4500 - val_loss: 3.6557 - val_accuracy: 0.4354
Epoch 7/100
4/4 [==============================] - 0s 59ms/step - loss: 2.3242 - accuracy: 0.4800 - val_loss: 3.6275 - val_accuracy: 0.4369
Epoch 8/100
4/4 [==

In [88]:
for obs in [100,250,500,1000,2000,5000]:
    transfer_cnn(obs,False)

Epoch 1/100
4/4 [==============================] - 0s 101ms/step - loss: 10.6136 - accuracy: 0.3000 - val_loss: 4.5651 - val_accuracy: 0.3934
Epoch 2/100
4/4 [==============================] - 0s 57ms/step - loss: 7.7709 - accuracy: 0.2900 - val_loss: 4.1246 - val_accuracy: 0.4052
Epoch 3/100
4/4 [==============================] - 0s 56ms/step - loss: 5.7543 - accuracy: 0.2700 - val_loss: 3.7888 - val_accuracy: 0.4170
Epoch 4/100
4/4 [==============================] - 0s 57ms/step - loss: 6.3170 - accuracy: 0.2700 - val_loss: 3.5374 - val_accuracy: 0.4185
Epoch 5/100
4/4 [==============================] - 0s 58ms/step - loss: 3.7700 - accuracy: 0.3500 - val_loss: 3.3438 - val_accuracy: 0.4177
Epoch 6/100
4/4 [==============================] - 0s 58ms/step - loss: 3.8244 - accuracy: 0.3200 - val_loss: 3.2043 - val_accuracy: 0.4162
Epoch 7/100
4/4 [==============================] - 1s 178ms/step - loss: 3.9208 - accuracy: 0.4000 - val_loss: 3.1236 - val_accuracy: 0.4162
Epoch 8/100
4/4 [

In [ ]:
test_xy(X_test_emb_de,y_test_de,'german',de_cnn2d_transfer)
test_xy(X_test_emb_fr,y_test_fr,'french',de_cnn2d_transfer)

2D trial

In [ ]:
class text_to_embed(object):
    '''
    takes text and embeddingmodel as input and outputs sequence of embeddings
    '''
    def __init__(self
                 , text = None
                 , embed_de = None
                 , embed_fr = None
                 , seq_len = None
                 , rep_dict = rep_dict
                 , embedding_dim=300):
                 
        self.text = text
        self.embed_de = embed_de
        self.embed_fr = embed_fr
        self.seq_len = seq_len
        self.embedding_dim = embedding_dim

    def t2s(self,line):
        sen_embed = np.zeros((self.embedding_dim,self.seq_len))
        words = line.split()
        for w in range(0,self.seq_len):
            try: 
                emb_fr = self.embed_fr[words[w]]
                emb_de = self.embed_de[words[w]]
                
            except:
                emb_de = np.zeros(embedding_dim)
                emb_fr = np.zeros(embedding_dim)
            #try: 
            #    emb_fr = self.embed_fr[words[w]]
            #except:
            #    emb_fr = np.zeros(embedding_dim)

            emb_de = np.expand_dims(emb_de, axis=0)
            emb_fr = np.expand_dims(emb_fr, axis=0)

            if w == 0 :
                sen_embed_de = emb_de
                sen_embed_fr = emb_fr
            else:
                sen_embed_de = np.concatenate([sen_embed_de,emb_de],axis=0)
                sen_embed_fr = np.concatenate([sen_embed_fr,emb_fr],axis=0)

        sen_emb = np.stack([sen_embed_de,sen_embed_fr],axis=0)
        return sen_emb #np.array(tokens)

    def t2s_old(self,line):
        #tokens = []
        sen_embed = np.zeros((self.embedding_dim,self.seq_len))
        words = line.split()
        for w in range(0,self.seq_len):
            try: 
              if la == 'de':
                  emb = self.embed_de[words[w]]
              elif la == 'fr':
                  emb = self.embed_fr[words[w]]
              else:
                  print(la)
                  emb = np.zeros(self.embedding_dim)
            except:
                emb = np.zeros(self.embedding_dim)
            #tokens.append(tok)
            sen_embed[:,w] = emb

        sen_embed = np.swapaxes(sen_embed,0,1)
        return sen_embed #np.array(tokens)

    def __iter__(self):
        for line in tqdm(self.text):
            line = self.t2s(line)
            yield line

In [ ]:
X_train_emb_de = np.array(list(text_to_embed(X_train_de,  de_git_embed, fr_git_embed, seq_len=seq_len)))
X_val_emb_de = np.array(list(text_to_embed(X_val_de, de_git_embed, fr_git_embed, seq_len=seq_len)))
X_test_emb_de = np.array(list(text_to_embed(X_test_de, de_git_embed, fr_git_embed, seq_len=seq_len)))

X_train_emb_fr = np.array(list(text_to_embed(X_train_fr, de_git_embed, fr_git_embed, seq_len=seq_len)))
X_val_emb_fr = np.array(list(text_to_embed(X_val_fr, de_git_embed, fr_git_embed, seq_len=seq_len)))
X_test_emb_fr = np.array(list(text_to_embed(X_test_fr, de_git_embed, fr_git_embed, seq_len=seq_len)))

In [ ]:
print('de',X_train_emb_de.shape, X_val_emb_de.shape, X_test_emb_de.shape)
print('fr',X_train_emb_fr.shape, X_val_emb_fr.shape, X_test_emb_fr.shape)

In [ ]:
X_train_emb_de = np.swapaxes(X_train_emb_de,2,3)
X_val_emb_de = np.swapaxes(X_val_emb_de,2,3)
X_test_emb_de = np.swapaxes(X_test_emb_de,2,3)

X_train_emb_fr = np.swapaxes(X_train_emb_fr,2,3)
X_val_emb_fr = np.swapaxes(X_val_emb_fr,2,3)
X_test_emb_fr = np.swapaxes(X_test_emb_fr,2,3)

In [ ]:
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,GlobalAveragePooling2D,Dropout,Flatten,AveragePooling2D
dropout_rate=.30

input_layer = Input(shape = (2,embedding_dim,seq_len,), name='text_input')
#pool_layer = AveragePooling2D(pool_size=(2, 1),name='avg_pooling2')(input_layer)
conv_layer = Conv2D(10, kernel_size=(100, 10), activation='relu', padding="same")(input_layer)
pool_layer = AveragePooling2D(pool_size=(1, 5),name='avg_pooling3')(conv_layer)
pool_layer = AveragePooling2D(pool_size=(2, 1),name='avg_pooling2')(pool_layer)
conv_layer = Conv2D(5, kernel_size=(50, 2), activation='relu', padding="same")(pool_layer)
#pool_layer = AveragePooling2D(pool_size=(1, 5),name='avg_pooling3')(conv_layer)
#conv_layer = Conv2D(50, kernel_size=(3, 3), activation='relu', padding="same")(conv_layer)
#conv_layer = AveragePooling2D(pool_size=(1, 3),name='avg_pooling3')(conv_layer)
#mask_layer = Masking(mask_value=0.)(input_layer)
#pool_layer = AveragePooling2D(pool_size=(2, 1),name='avg_pooling')(conv_layer)
pool_layer = Flatten()(conv_layer)
#pool_layer = GlobalAveragePooling2D(name='avg_pooling')(pool_layer)
dens_layer = Dense(150, activation='relu',name='dense')(pool_layer)
drop_layer = Dropout(dropout_rate)(pool_layer)
pred_layer_3 = Dense(no_Classes, activation = 'softmax', name='out')(drop_layer) 

de_avg_pool = Model(inputs = [input_layer], outputs = [pred_layer_3])

lr = .001
opt = Adam(lr=lr, decay=lr/200)
de_avg_pool.summary()
de_avg_pool.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)

hist = de_avg_pool.fit(x = X_train_emb_de, y = y_train_enc_de,\
                validation_data = (X_val_emb_de, y_val_enc_de), \
                epochs = 100, batch_size = 256, shuffle = True, \
                class_weight = class_weight_dict_de, \
                callbacks = [early_stopping])

In [ ]:
import json
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,balanced_accuracy_score,confusion_matrix
import numpy as np

def test_xy(X,y,string,model):
    y_pred = model.predict([X])
    y_pred_arg = y_pred.argmax(axis=1)
    pred= [encoder.classes_[y] for y in y_pred_arg]
    print('accuracy %s'% string,accuracy_score(pred,y))
    print('b_accuracy %s'%  string,balanced_accuracy_score(pred, y))
    #return pred, y

#test_xy(X_test_emb_de,y_test_de,'together')
test_xy(X_test_emb_de,y_test_de,'german',de_avg_pool)
test_xy(X_test_emb_fr,y_test_fr,'french',de_avg_pool)
#print(classification_report(y,pred))

aa

# Multilingual Training

## CNN

In [ ]:
X_train_emb =  np.concatenate((X_train_emb_de, X_train_emb_fr))
y_train_enc =  np.concatenate((y_train_enc_de, y_train_enc_fr))
X_val_emb =  np.concatenate((X_val_emb_de, X_val_emb_fr))
y_val_enc =  np.concatenate((y_val_enc_de, y_val_enc_fr))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Bidirectional, GlobalAveragePooling1D, Concatenate,LSTM,BatchNormalization,Add,Masking,GlobalMaxPooling1D,Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

dropout_rate=.7

input_imbd_1 = Input(shape = (seq_len,embedding_dim,), name='text_input')
mask_layer = Masking(mask_value=0.)(input_imbd_1)
conv_layer = Conv1D(filters=300,   kernel_size= 10,   padding='valid',  activation='relu', strides=1)(input_imbd_1)
conv_layer2 = Conv1D(filters=100,   kernel_size= 5,   padding='valid',  activation='relu', strides=2)(input_imbd_1)
#lstm_layer_1 = Bidirectional(LSTM(128,activation = 'tanh',recurrent_activation = 'sigmoid', dropout = dropout_rate,recurrent_dropout = 0, unroll = False,use_bias = True))(conv_layer)
lstm_layer_1 = GlobalAveragePooling1D()(conv_layer)
#lstm_layer_1 = GlobalMaxPooling1D()(conv_layer)
dens_layer_1 = Dense(100, activation='relu')(lstm_layer_1)
drop_layer_1 = Dropout(dropout_rate)(dens_layer_1)
pred_layer_3 = Dense(74, activation = 'softmax', name='o3')(drop_layer_1) 

de_cnn_avg = Model(inputs = [input_imbd_1], outputs = [pred_layer_3])

lr = .001
opt = Adam(lr=lr, decay=lr/100)

de_cnn_avg.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
early_stopping = EarlyStopping(patience = 5)
de_cnn_avg.summary()
hist = de_cnn_avg.fit(x = X_train_emb, y = y_train_enc,\
                validation_data = (X_val_emb, y_val_enc), \
                epochs = 150, batch_size = 64, shuffle = True, \
                class_weight = class_weight_dict_de_fr, \
                callbacks = [early_stopping])

In [ ]:
import json
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

def test_xy(X,y,string):
    y_pred = de_cnn_avg.predict([X])
    y_pred_arg = y_pred.argmax(axis=1)
    pred= [encoder.classes_[y] for y in y_pred_arg]
    print('accuracy %s'% string,accuracy_score(pred,y))
    print('b_accuracy %s'%  string,balanced_accuracy_score(pred, y))

test_xy(X_test_emb_de,y_test_de,'german')
test_xy(X_test_emb_fr,y_test_fr,'french')
#print(classification_report(pred, df_test['cc5']))

## CNN with training embedding ayer

In [ ]:
 from tensorflow.keras.layers import BatchNormalization

 def run_ml_CNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,
             vocab_size,embedding_matrix,class_weight_dict,no):
   
    X_train_pad =  np.concatenate((X_train_pad_de, X_train_pad_fr[:no]))
    y_train_enc =  np.concatenate((y_train_enc_de, y_train_enc_fr[:no]))

    imbala = int(len(X_train_pad_de)/no)
    for n in range(1,imbala):
        X_train_pad =  np.concatenate((X_train_pad, X_train_pad_fr[:no]))
        y_train_enc =  np.concatenate((y_train_enc, y_train_enc_fr[:no]))
    X_val_pad =  np.concatenate((X_val_pad_de, X_val_pad_fr))
    y_val_enc =  np.concatenate((y_val_enc_de, y_val_enc_fr))
    print(len(X_train_pad))

    dropout_rate=.7
    lr = .0003
    opt = Adam(lr=lr, decay=lr/100)

    input_layer = Input(shape = (seq_len,), dtype = 'float')
    embedd_seq = Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], input_length = seq_len, trainable = True, mask_zero=False)(input_layer)
    norma_layer = BatchNormalization()(embedd_seq)
    convo_layer = Conv1D(filters=300,   kernel_size= 5,   padding='valid',  activation='relu', strides=1)(norma_layer)
    norma_layer = BatchNormalization()(convo_layer)
    pool_layer = GlobalMaxPooling1D()(norma_layer)
    #pool_layer = GlobalAveragePooling1D()(convo_layer)
    dens_layer_1 = Dense(150, activation='relu')(pool_layer)
    drop_layer = Dropout(dropout_rate)(dens_layer_1)
    pred_layer = Dense(y_train_enc.shape[1], activation = 'softmax')(drop_layer) #, kernel_regularizer=l1_l2(0.001)

    cnn_de_fr = Model(inputs = [input_layer], outputs = pred_layer)
    cnn_de_fr.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = cnn_de_fr.fit(x = X_train_pad, y = y_train_enc,\
                    validation_data = (X_val_pad, y_val_enc), \
                    epochs = 50, batch_size = 256, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test_de = cnn_de_fr.predict(X_test_pad_de)
    y_pred_arg_de = y_pred_test_de.argmax(axis=1)
    y_pred_test_de = [encoder.classes_[y] for y in y_pred_arg_de]

    y_pred_test_fr = cnn_de_fr.predict(X_test_pad_fr)
    y_pred_arg_fr = y_pred_test_fr.argmax(axis=1)
    y_pred_test_fr= [encoder.classes_[y] for y in y_pred_arg_fr]

    print('de test obs.',no,'accuracy %s' % accuracy_score(y_pred_test_de, y_test_de))
    print('de test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test_de, y_test_de))
    print('fr test obs.',no,'accuracy %s' % accuracy_score(y_pred_test_fr, y_test_fr))
    print('fr test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test_fr, y_test_fr))


In [ ]:
for obs in [500,1000,2000,5000]:
     run_ml_CNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,
             vocab_size_de,embedding_matrix_de,class_weight_dict_de,obs)


In [ ]:
run_CNN_best(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,vocab_size_de,embedding_matrix_de,class_weight_dict_de)

In [ ]:


dropout_rate=.6
lr = .001
opt = Adam(lr=lr, decay=lr/100)

input_layer = Input(shape = (seq_len,), dtype = 'float')
embedd_seq = Embedding(vocab_size_de_fr, embedding_dim, weights = [embedding_matrix_de_fr], input_length = seq_len, trainable = True, mask_zero=False)(input_layer)
conv_layer = Conv1D(filters=100,   kernel_size= 5,   padding='valid',  activation='relu', strides=1)(embedd_seq)
pool_layer = GlobalMaxPooling1D(name='max_pool')(conv_layer)
#pool_layer = GlobalAveragePooling1D(name='avg_pooling')(conv_layer)
dens_layer = Dense(150, activation='tanh',name='dense')(pool_layer)
drop_layer = Dropout(dropout_rate)(dens_layer)
pred_layer = Dense(y_train_enc.shape[1], activation = 'softmax')(drop_layer) #, kernel_regularizer=l1_l2(0.001)

cnn_max_pool_mod = Model(inputs = [input_layer], outputs = pred_layer)
cnn_max_pool_mod.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
#print(Ftext_pool.summary())
early_stopping = EarlyStopping(patience = 5)

hist = cnn_max_pool_mod.fit(x = X_train_pad, y = y_train_enc,\
                validation_data = (X_val_pad, y_val_enc), \
                epochs = 50, batch_size = 256, shuffle = True, class_weight = class_weight_dict_de_fr, \
                callbacks = [early_stopping])

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred_t_de = cnn_max_pool_mod.predict(X_test_pad_de)
y_pred_arg = y_pred_t_de.argmax(axis=1)
y_pred_t_de= [encoder.classes_[y] for y in y_pred_arg]

y_pred_t_fr = cnn_max_pool_mod.predict(X_test_pad_fr)
y_pred_arg = y_pred_t_fr.argmax(axis=1)
y_pred_t_fr= [encoder.classes_[y] for y in y_pred_arg]

print('train obs.','accuracy %s' % accuracy_score(y_pred_t_de, y_test_de))
print('train obs.','b_accuracy %s' % balanced_accuracy_score(y_pred_t_de, y_test_de))
print('train obs.','accuracy %s' % accuracy_score(y_pred_t_fr, y_test_fr))
print('train obs.','b_accuracy %s' % balanced_accuracy_score(y_pred_t_fr, y_test_fr))


## CNN2D German - French

In [ ]:
 def run_ml_CNN2D(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,
             vocab_size,embedding_matrix,class_weight_dict,no):
   
    X_train_pad =  np.concatenate((X_train_pad_de, X_train_pad_fr[:no]))
    y_train_enc =  np.concatenate((y_train_enc_de, y_train_enc_fr[:no]))

    imbala = int(len(X_train_pad_de)/no)
    for n in range(1,imbala):
        X_train_pad =  np.concatenate((X_train_pad, X_train_pad_fr[:no]))
        y_train_enc =  np.concatenate((y_train_enc, y_train_enc_fr[:no]))
    X_val_pad =  np.concatenate((X_val_pad_de, X_val_pad_fr))
    y_val_enc =  np.concatenate((y_val_enc_de, y_val_enc_fr))
    print(imbala,len(X_train_pad_de),len(X_train_pad))

    dropout_rate=.4
    filter_sizes = [1,2,3,4,5]
    num_filters = 15

    input_layer = Input(shape = (seq_len,), name='text_input')
    x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable = True)(input_layer)
    x = Reshape((seq_len, embedding_dim, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_dim),
                                      kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(seq_len - filter_sizes[i] + 1, 1))(conv))

    pool_layer = Concatenate(axis=1)(maxpool_pool)   

    falt_layer = Flatten(name='flat')(pool_layer)
    drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn2d_ml = Model(inputs = [input_layer], outputs = [pred_layer])

    lr = .005
    opt = Adam(lr=lr, decay=lr/100)

    de_cnn2d_ml.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = de_cnn2d_ml.fit(x = X_train_pad, y = y_train_enc,\
                    validation_data = (X_val_pad, y_val_enc), \
                    epochs = 50, batch_size = 256, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test_de = de_cnn2d_ml.predict(X_test_pad_de)
    y_pred_arg_de = y_pred_test_de.argmax(axis=1)
    y_pred_test_de = [encoder.classes_[y] for y in y_pred_arg_de]

    y_pred_test_fr = de_cnn2d_ml.predict(X_test_pad_fr)
    y_pred_arg_fr = y_pred_test_fr.argmax(axis=1)
    y_pred_test_fr= [encoder.classes_[y] for y in y_pred_arg_fr]

    print('de test obs.',no,'accuracy %s' % accuracy_score(y_pred_test_de, y_test_de))
    print('de test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test_de, y_test_de))
    print('fr test obs.',no,'accuracy %s' % accuracy_score(y_pred_test_fr, y_test_fr))
    print('fr test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test_fr, y_test_fr))

In [ ]:
for obs in [100,250,500,1000,2000,5000]:
     run_ml_CNN2D(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,
             vocab_size_de,embedding_matrix_de,class_weight_dict_de,obs)

## CNN2D French - German

In [ ]:
 def run_ml_CNN2D(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,
             vocab_size,embedding_matrix,class_weight_dict,no):
   
    X_train_pad =  np.concatenate((X_train_pad_de[:no], X_train_pad_fr))
    y_train_enc =  np.concatenate((y_train_enc_de[:no], y_train_enc_fr))

    imbala = int(len(X_train_pad_fr)/no)
    if imbala > 1:
        for n in range(1,imbala):
            X_train_pad =  np.concatenate((X_train_pad, X_train_pad_de[:no]))
            y_train_enc =  np.concatenate((y_train_enc, y_train_enc_de[:no]))
    X_val_pad =  np.concatenate((X_val_pad_de, X_val_pad_fr))
    y_val_enc =  np.concatenate((y_val_enc_de, y_val_enc_fr))
    print(imbala,len(X_train_pad_de),len(X_train_pad))

    dropout_rate=.5
    filter_sizes = [1,2,3,4,5]
    num_filters = 20

    input_layer = Input(shape = (seq_len,), name='text_input')
    x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable = True)(input_layer)
    x = Reshape((seq_len, embedding_dim, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_dim),
                                      kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(seq_len - filter_sizes[i] + 1, 1))(conv))

    pool_layer = Concatenate(axis=1)(maxpool_pool)   

    falt_layer = Flatten(name='flat')(pool_layer)
    drop_layer = Dropout(dropout_rate,name='drop')(falt_layer)
    pred_layer = Dense(no_Classes, activation = 'softmax', name='output_de')(drop_layer) 

    de_cnn2d_ml = Model(inputs = [input_layer], outputs = [pred_layer])

    lr = .005
    opt = Adam(lr=lr, decay=lr/100)

    de_cnn2d_ml.compile(optimizer = opt, loss = ['categorical_crossentropy'], metrics = ['accuracy'])
    early_stopping = EarlyStopping(patience = 5)

    hist = de_cnn2d_ml.fit(x = X_train_pad, y = y_train_enc,\
                    validation_data = (X_val_pad, y_val_enc), \
                    epochs = 50, batch_size = 256, shuffle = True, class_weight = class_weight_dict, \
                    callbacks = [early_stopping])
    
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    y_pred_test_de = de_cnn2d_ml.predict(X_test_pad_de)
    y_pred_arg_de = y_pred_test_de.argmax(axis=1)
    y_pred_test_de = [encoder.classes_[y] for y in y_pred_arg_de]

    y_pred_test_fr = de_cnn2d_ml.predict(X_test_pad_fr)
    y_pred_arg_fr = y_pred_test_fr.argmax(axis=1)
    y_pred_test_fr= [encoder.classes_[y] for y in y_pred_arg_fr]

    print('de test obs.',no,'accuracy %s' % accuracy_score(y_pred_test_de, y_test_de))
    print('de test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test_de, y_test_de))
    print('fr test obs.',no,'accuracy %s' % accuracy_score(y_pred_test_fr, y_test_fr))
    print('fr test obs.',no,'b_accuracy %s' % balanced_accuracy_score(y_pred_test_fr, y_test_fr))

In [ ]:
for obs in [100,250,500,1000,2000,5000,10000,15000]:
     run_ml_CNN2D(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,
             X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,
             vocab_size_de,embedding_matrix_de,class_weight_dict_de,obs)

## Best Model

In [ ]:
X_train_pad =  np.concatenate((X_train_pad_de, X_train_pad_fr))
y_train_enc =  np.concatenate((y_train_enc_de, y_train_enc_fr))
X_val_pad =  np.concatenate((X_val_pad_de, X_val_pad_fr))
y_val_enc =  np.concatenate((y_val_enc_de, y_val_enc_fr))

In [ ]:
run_CNN_best(X_train_pad_fr,y_train_enc_fr,X_val_pad_fr,y_val_enc_fr,X_test_pad_fr,y_test_fr,vocab_size_fr,embedding_matrix_fr,class_weight_dict_fr)

## CNN Zero Shot with training embedding layer

In [ ]:
dropout_rate=.7
lr = .001
opt = Adam(lr=lr, decay=lr/100)

input_layer = Input(shape = (seq_len,), dtype = 'float')
embedd_seq = Embedding(vocab_size_de_fr, embedding_dim, weights = [embedding_matrix_de_fr], input_length = seq_len, trainable = True, mask_zero=False)(input_layer)
conv_layer = Conv1D(filters=150,   kernel_size= 7,   padding='valid',  activation='relu', strides=1)(embedd_seq)
pool_layer = GlobalMaxPooling1D(name='max_pool')(conv_layer)
drop_layer = Dropout(dropout_rate)(pool_layer)
pred_layer = Dense(y_train_enc.shape[1], activation = 'softmax')(drop_layer) #, kernel_regularizer=l1_l2(0.001)

cnn_max_pool_mod = Model(inputs = [input_layer], outputs = pred_layer)
cnn_max_pool_mod.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
#print(Ftext_pool.summary())
early_stopping = EarlyStopping(patience = 5)

hist = cnn_max_pool_mod.fit(x = X_train_pad_de, y = y_train_enc_de,\
                validation_data = (X_val_pad_de, y_val_enc_de), \
                epochs = 50, batch_size = 256, shuffle = True, class_weight = class_weight_dict_de_fr, \
                callbacks = [early_stopping])

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred_t_de = cnn_max_pool_mod.predict(X_test_pad_de)
y_pred_arg = y_pred_t_de.argmax(axis=1)
y_pred_t_de= [encoder.classes_[y] for y in y_pred_arg]

y_pred_t_fr = cnn_max_pool_mod.predict(X_test_pad_fr)
y_pred_arg = y_pred_t_fr.argmax(axis=1)
y_pred_t_fr= [encoder.classes_[y] for y in y_pred_arg]

print('train obs.','accuracy %s' % accuracy_score(y_pred_t_de, y_test_de))
print('train obs.','b_accuracy %s' % balanced_accuracy_score(y_pred_t_de, y_test_de))
print('train obs.','accuracy %s' % accuracy_score(y_pred_t_fr, y_test_fr))
print('train obs.','b_accuracy %s' % balanced_accuracy_score(y_pred_t_fr, y_test_fr))

# Bert

In [ ]:
%%capture
!pip install ktrain

In [ ]:
class_names_ = df_de['cc5'].unique()
class_names_.sort()

In [ ]:
#run_CNN(X_train_pad_de,y_train_enc_de,X_val_pad_de,y_val_enc_de,X_test_pad_de,y_test_de,vocab_size_de,embedding_matrix_de,class_weight_dict_de,obs)


In [ ]:
var1 = 'categ'
var2 = 'words_from_url'
var3 = 'name'
df_fr['text'] = 'fr ' + df_fr[var1].fillna('unknown')  + ' . ' + df_fr[var2].fillna('unknown')  + ' . ' + df_fr[var3].fillna('unknown') 
df_de['text'] = 'de ' + df_de[var1].fillna('unknown')  + ' . ' + df_de[var2].fillna('unknown')  + ' . ' + df_de[var3].fillna('unknown') 
df_at['text'] = 'de ' + df_at[var1].fillna('unknown')  + ' . ' + df_at[var2].fillna('unknown')  + ' . ' + df_at[var3].fillna('unknown') 

from sklearn.model_selection import train_test_split
def split_train_abs(df):
    X_train, X_val_test, y_train, y_val_test  = train_test_split(df['text'], df['cc5'], random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_val_test , y_val_test, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

X_train_de, X_val_de, X_test_de, y_train_de, y_val_de, y_test_de = split_train_abs(df_de)
X_train_fr, X_val_fr, X_test_fr, y_train_fr, y_val_fr, y_test_fr = split_train_abs(df_fr)

print('de',X_train_de.shape, X_val_de.shape, X_test_de.shape, y_train_de.shape, y_val_de.shape, y_test_de.shape )
print('fr',X_train_fr.shape, X_val_fr.shape, X_test_fr.shape, y_train_fr.shape, y_val_fr.shape, y_test_fr.shape )

X_train_de = [str(sen) for sen in X_train_de]
X_val_de = [str(sen) for sen in X_val_de]
X_test_de = [str(sen) for sen in X_test_de]

y_train_de = [str(sen) for sen in y_train_de]
y_val_de = [str(sen) for sen in y_val_de]
y_test_de = [str(sen) for sen in y_test_de]

X_train_fr = [str(sen) for sen in X_train_fr]
X_val_fr = [str(sen) for sen in X_val_fr]
X_test_fr = [str(sen) for sen in X_test_fr]

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

encoder = LabelEncoder()
encoder.fit(df_de['cc5'])

def encode_label(y_):
    y__ = encoder.transform(y_)
    #y_enc =tf.keras.utils.to_categorical(y__, num_classes=no_Classes, dtype="float32")
    return y__

y_train_enc_de = encode_label(y_train_de)
y_val_enc_de = encode_label(y_val_de)
y_test_enc_de = encode_label(y_test_de)

y_train_enc_fr = encode_label(y_train_fr)
y_val_enc_fr = encode_label(y_val_fr)
y_test_enc_fr = encode_label(y_test_fr)

print(y_train_enc_de.shape,y_val_enc_de.shape,y_test_enc_de.shape)
print(y_train_enc_fr.shape,y_val_enc_fr.shape,y_test_enc_fr.shape)

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'xlm-roberta-large'#'bert-base-multilingual-cased'
t = text.Transformer(MODEL_NAME, maxlen=60, class_names=class_names_)

trn = t.preprocess_train(X_train_de, y_train_de)
val = t.preprocess_test(X_val_de, y_val_de)
#test = t.preprocess_test(X_test_de, y_test_enc_de)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)


In [ ]:
learner.lr_find()
learner.lr_plot()

In [ ]:
learner.fit(5e-5, 5)

In [ ]:
%%capture
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_data = pd.DataFrame(list(zip(X_train_de,y_train_de)), columns =['text','y'])
eval_df = pd.DataFrame(list(zip(X_val_de,y_val_de)), columns =['text','y'])
test_df = pd.DataFrame(list(zip(X_test_de,y_test_de)), columns =['text','y'])

 
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
#train_data = [['Example sentence belonging to class 1', 1], ['Example sentence belonging to class 0', 0]]
#train_df = pd.DataFrame(train_data)

#eval_data = [['Example eval sentence belonging to class 1', 1], ['Example eval sentence belonging to class 0', 0]]
#eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model = ClassificationModel('XLM,', 'XLM-RoBERTa') # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

get the french

In [ ]:
len(class_names)
Y_FRENCH.shape

In [ ]:
X_FRENCH = list(df_fr['text'])
Y_FRENCH = encode_label(df_fr['cc5'],encoder5)

test = t.preprocess_test(X_FRENCH, Y_FRENCH)
learner.validate(val_data=test, class_names=t.get_classes())

In [ ]:
reloaded_predictor = ktrain.get_predictor(learner.model, preproc=preproc)
reloaded_predictor.predict(X_test[4])

In [ ]:
!pip install transformers
!pip install tensorflow==2.1.0

In [ ]:
import torch
import tensorflow as tf
from transformers import AutoModel, AutoTokenizer, BertTokenizer

torch.set_grad_enabled(False)

In [ ]:
# Store the model we want to use
MODEL_NAME = "bert-base-multilingual-cased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
from transformers import (TFBertForSequenceClassification, 
                          BertTokenizer,
                          TFRobertaForSequenceClassification, 
                          RobertaTokenizer)

bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

#roberta_model = TFRobertaForSequenceClassification.from_pretrained("roberta-base")
#roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
sequence = "Systolic arrays are cool. This 🐳 is cool too."

bert_tokenized_sequence = bert_tokenizer.tokenize(sequence)

print("BERT:", bert_tokenized_sequence)

In [ ]:

encoded_bert_sequence = bert_tokenizer.encode(list(X_train_de), add_special_tokens=True, max_length=128)



In [ ]:
df_train = pd.DataFrame(zip(X_train_de , y_train_de),columns=['text','cc5'])
df_train = pd.DataFrame(zip(X_train_de , y_train_de),columns=['text','cc5'])
df_train = pd.DataFrame(zip(X_train_de , y_train_de),columns=['text','cc5'])

In [ ]:
encoded_bert_sequence = bert_tokenizer.encode(list(X_train_de), add_special_tokens=True, max_length=64)

tf_train = tf.data.Dataset.from_tensor_slices((encoded_bert_sequence, df_train['cc5'].values))

In [ ]:
from transformers import glue_convert_examples_to_features

bert_train_dataset = glue_convert_examples_to_features(tf_train, bert_tokenizer, 64, 'mrpc')

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [ ]:
print("Fine-tuning BERT on MRPC")
bert_history = bert_model.fit(bert_train_dataset, epochs=3, validation_data=bert_validation_dataset)


In [ ]:
print("Evaluating the BERT model")
bert_model.evaluate(bert_validation_dataset)
